##   SENTIMENT ANALYSIS

In [ ]:
We will be doing a sentimnent analysis where we will be also deploying and using our model using gradio

In [1]:
import numpy as np
import pandas as pd


In [2]:
df =pd.read_csv('samples.csv')

In [3]:
df.shape

(4000, 2)

In [134]:
df.head()

,sentiment,text
0,Negative,why does google criminalize asian women by gen...
1,Positive,watching the streamers tear the bags build the...
2,Negative,mfs be 17 in the training facilities of fifa b...
3,Negative,inm watching reviews of the old assassins cree...
4,Negative,narendramodi pmoindia please help me twitter...


In [138]:
df = df[df.sentiment != "Irrelevant"]

For the data that we will need to process there should not be null values

In [140]:
df.isnull().sum()

sentiment    0
text         3
dtype: int64

In [141]:
df.dropna(inplace=True)

In the below code we will be encoding our data and dataset where we will map our positive values to 1 and our negative values to 0

In [142]:
df = df[df['sentiment'].isin(['Positive', 'Negative'])]
df['sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': 0})



In [143]:
unique_sentiments = df['sentiment'].unique()
print(unique_sentiments)


[0 1]


In [147]:
df.head()

,sentiment,text
0,0,why does google criminalize asian women by gen...
1,1,watching the streamers tear the bags build the...
2,0,mfs be 17 in the training facilities of fifa b...
3,0,inm watching reviews of the old assassins cree...
4,0,narendramodi pmoindia please help me twitter...


When executing the below code we are  turning the text from uppercase to lowercase

In [130]:

df['text'] = df['text'].str.lower()

In the below code we will be looking at the distrbution of our target features and we can see that our target variable is close to balanced

In [149]:
df['sentiment'].value_counts()

0    1693
1    1399
Name: sentiment, dtype: int64

For this below line of code we are splitting our dataset to our training and test data  for training our model

In [151]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)


In [152]:
print('Number of train data points:',len(X_train))
print('Number of train labels:',len(y_train))
print('Number od test data points:',len(X_test))
print('Number of test labels:',len(y_test))

Number of train data points: 2473
Number of train labels: 2473
Number od test data points: 619
Number of test labels: 619


In [154]:
pip install transformers



     ---------------------------------------- 7.2/7.2 MB 71.3 kB/s eta 0:00:00
     ------------------------------------ 263.9/263.9 kB 463.7 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 190.4 kB/s eta 0:00:00
     ------------------------------------ 268.8/268.8 kB 147.8 kB/s eta 0:00:00


In the below codethe DistilBertTokenizerFast class is imported from the transformers library. Then, the tokenizer instance is created using the from_pretrained method, specifying the pre-trained model distilbert-base-uncased-finetuned-sst-2-english. This tokenizer can be used to tokenize and preprocess text for tasks such as sentiment analysis or text classification using the DistilBERT model.

In [173]:
from  transformers  import DistilBertTokenizerFast
models ='distilbert-base-uncased-finetuned-sst-2-english'
tokenizer =DistilBertTokenizerFast.from_pretrained(model)

In [160]:
print('text:', X_train.iloc[0])
tokenizer(X_train.iloc[0], truncation=True, padding=True, max_length=256, return_tensors="tf")


text: chile verizon said fuck the


{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[  101,  7029,  2310, 21885,  2239,  2056,  6616,  1996,   102]])>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [163]:
print(X_train)


205                           chile verizon said fuck the
3339    i dont know what im doing wrong but the rogue ...
3987    7 took a little over a full month and to get e...
2507    i  m already beginning to get think that this ...
3582                        impo ant research and so much
                              ...                        
2095                 psas clean paladin is still terrible
1394    why the fuck is nobody playing rush on battlef...
1438                       after taking a few games nba2k
1640                     cuz i love pubg more than my bae
1107    bruh the sony reveal was pointless did we get ...
Name: text, Length: 2473, dtype: object


The code below is using the `tokenizer` object to encode and preprocess text data using the Hugging Face `transformers` library. It assumes that you have defined the `tokenizer` object and set the `MAX_LENGTH` variable to 256. 

The code then applies the tokenizer to the training and test data (`X_train` and `X_test`). It performs the following steps:


- `tokenizer`: Calls the tokenizer object.
- `X_train.tolist()`: Converts the training data to a list if it's not already in list format.
- `truncation=True`: Truncates the input sequences to the specified `MAX_LENGTH` if they exceed it.
- `padding=True`: Adds padding to the sequences to ensure they have the same length.
- `return_tensors="tf"`: Returns TensorFlow tensors as the output format.
- `max_length=MAX_LENGTH`: Specifies the maximum length of the tokenized sequences.

The steps for encoding the test data are the same as for the training data, but it applies the tokenizer to the `X_test` data.

After running this code, `train_encodings` and `test_encodings` will contain the encoded and preprocessed representations of the training and test data, respectively, suitable for input to a downstream machine learning model, such as a BERT-based classifier or sequence labeling model.

In [164]:
MAX_LENGTH = 256

train_encodings = tokenizer(
    X_train.tolist(),
    truncation=True,
    padding=True,
    return_tensors="tf",
    max_length=MAX_LENGTH
)

test_encodings = tokenizer(
    X_test.tolist(),
    truncation=True,
    padding=True,
    return_tensors="tf",
    max_length=MAX_LENGTH
)


In [167]:
import tensorflow as tf

In [169]:
train =tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                         y_train))
test =tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                       y_test))

In [174]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=models,
    num_labels=2
)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [175]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=5e-5
)


In [177]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])


In [178]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In the below line of code we have two epochs where we have a validation acucuracy of 80 percent

In [182]:
BATCH_SIZE =16
NUM_EPOCHS =2
model.fit(train.shuffle(len(X_train)).batch(BATCH_SIZE),
         epochs =NUM_EPOCHS,
         batch_size=BATCH_SIZE,
         validation_data=test.shuffle(len(X_test)).batch(BATCH_SIZE))

Epoch 1/2
155/155 [==============================] - 1639s 11s/step - loss: 0.3781 - accuracy: 0.8439 - val_loss: 0.4262 - val_accuracy: 0.8061
Epoch 2/2
155/155 [==============================] - 1492s 10s/step - loss: 0.1913 - accuracy: 0.9232 - val_loss: 0.4605 - val_accuracy: 0.8094


In [197]:
model.save_pretrained('/sentiment_model')

In [209]:
unseen_data=df.sample(frac=0.05,random_state =42)
unseen_reviews =unseen_data['text'].tolist()
unseen_encodings =tokenizer(unseen_reviews,
                           padding=True,
                           truncation=True,
                           max_length =MAX_LENGTH,
                           return_tensors ="tf")
y_unseen =unseen_data['sentiment'].tolist()
unseen_encodings=tf.data.Dataset.from_tensor_slices((dict(unseen_encodings),
                                                   y_unseen))

In [190]:
label_mapping = {'positive': 1, 'negative': 0}

# Convert string labels to numeric values with a default value of -1 if label is not found
y_train = [label_mapping.get(label, -1) for label in y_train]
y_test = [label_mapping.get(label, -1) for label in y_test]

# Filter out the examples with unknown labels
y_train = [label for label in y_train if label != -1]
y_test = [label for label in y_test if label != -1]


In [205]:
import numpy as np

label_mapping = {"label1": 0, "label2": 1, "label3": 2}
default_label = -1

# Convert string labels to integers
y_unseen = [label_mapping.get(label, default_label) for label in y_unseen]



In [211]:
model.evaluate(unseen_encodings.shuffle(len(unseen_reviews)).batch(BATCH_SIZE),
               return_dict=True,
               batch_size=BATCH_SIZE)


10/10 [==============================] - 24s 2s/step - loss: 0.4284 - accuracy: 0.8000


{'loss': 0.4284231960773468, 'accuracy': 0.800000011920929}

In [192]:
pip install gardio

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gardio (from versions: none)
ERROR: No matching distribution found for gardio


In [195]:
pip install gradio


     ---------------------------------------- 19.8/19.8 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 288.8/288.8 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 471.5/471.5 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 75.4/75.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 87.5/87.5 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 45.7/45.7 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 195.5/195.5 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 50.5/50.5 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 323.6/323.6 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 124.7/124.7 kB 1.8 MB/s eta 0:00:00
  Using cached mdurl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.7.1 which is incompatible.
tensorflow-recommenders 0.7.3 requires tensorflow>=2.9.0; sys_platform != "darwin", but you have tensorflow 2.6.0 which is incompatible.
tensorflow-intel 2.12.0 requires absl-py>=1.0.0, but you have absl-py 0.15.0 which is incompatible.
tensorflow-intel 2.12.0 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.6.0 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.19.5 which is incompatible.
tensorflow-intel 2.12.0 requires protobuf!=4.

In [216]:
pip install --upgrade typing_extensions


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade typing_extensions


In [219]:
pip uninstall typing_extensions


^C
Note: you may need to restart the kernel to use updated packages.


In [220]:
import gradio as gr
def predict(sentence):
    load_model =TFDistilBertForSequenceClassification.from_pretrained('/sentiment_model')
    predict_input =tokenizer.encode(sentence,
                                   truncation =True,
                                   padding=True,
                                   return_tensors ='tf')
    tf_output=load.predict(predict_input)[0]
    tf_prediction =tf.nn.softmax(tf_output,axis=1).numpy()[0]
    return['negative','positive'][np.argmax(tf-prediction)]
demo =gr.Interface(fn=predict, inputs ='text',outputs='text',live=True)

demo.launch

C:\Users\stilinski\anaconda3\New folder\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


<bound method Interface.launch of Gradio Interface for: predict
-----------------------------
inputs:
|-Textbox(label="None")
outputs:
|-Textbox(label="None")>

In [221]:
import gradio as gr

def predict(sentence):
    # Load your model and perform the prediction
    # Replace this with your actual model code
    prediction = "This is the predicted output for: " + sentence

    return prediction

# Create an input component for the sentence input
input_text = gr.inputs.Textbox(label="Enter a sentence")

# Create an output component to display the prediction
output_text = gr.outputs.Textbox(label="Prediction")

# Create the Gradio interface with the predict function, input, and output components
gr.Interface(fn=predict, inputs=input_text, outputs=output_text).launch()


IMPORTANT: You are using gradio version 2.0.4, however version 3.14.0 is available, please upgrade.
--------
Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)

[2023-07-12 13:55:24,897] ERROR in app: Exception on /static/bundle.css [GET]
Traceback (most recent call last):
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\stilinski\anaconda3\New folder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request


In the below line of code we have the gradio implementation of the model and using google colabs we would have actually have had two columns of text and  the sentiment .Thus that  is the breakdown of our asentiment analysis using deep learning 